In [1]:
# import the deliverables
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# read images from path 

# convert the data/images into a pd df

In [ ]:
# create the model using Sequential()

In [ ]:
# compile the model

In [ ]:
# model summary

In [ ]:
# train the model

In [ ]:
# visualize training results

In [ ]:
# predict on new data